<a href="https://colab.research.google.com/github/dtabuena/ApoE_Data_Clustering/blob/main/MaxLikelihood_Hipp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install openpyxl
!pip install XlsxWriter
!pip install pingouin
!pip install CMH
!pip install svgutils

from IPython.display import clear_output
import pandas as pd
import os
import numpy as np
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns

clear_output()

from sklearn.cluster import KMeans
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import pingouin as pg
import scipy
# from pandas.compat.numpy import np_array_datetime64_compat
from random import sample
from tqdm import tqdm
import os
import shutil
from google import colab


!wget 'https://www.dropbox.com/s/j0ts1hpt76xzvdq/arial.ttf'
mpl.font_manager.fontManager.addfont('/content/arial.ttf')

clear_output()




def restore_order(arg_ord):
    return np.argsort(arg_ord)

In [10]:
def get_files(link):
    my_drop_folder = 'my_drop_folder'
    zipped_file_path = "/content/"+my_drop_folder + ".zip"
    unzipped_file_path = "/content/"+my_drop_folder
    if not( os.path.exists(zipped_file_path)):
        !wget -O $zipped_file_path $link    # download with new name
    !echo A | unzip $zipped_file_path -d $unzipped_file_path
    file_list = [f for r,d,f in os.walk("/content/"+my_drop_folder )][0]
    return file_list

def read_data_file(xl_to_analyze):
    my_df_dict  = pd.read_excel('my_drop_folder/'+xl_to_analyze, engine='openpyxl',index_col=None,sheet_name=None)

    NEW_my_df_dict = {}
    for k in my_df_dict.keys():
        new_k = k
        if 'cre' in new_k:
            new_k = k.replace('Syn1-cre','/Syn1-Cre')
        NEW_my_df_dict[new_k] = my_df_dict[k]

    return NEW_my_df_dict, xl_to_analyze

def dict_to_df(my_df_dict,AP_cut=60):
    '''convert dict of categories into a DF'''
    for k in my_df_dict.keys():
        df = my_df_dict[k]
        df['type'] = k
    df_list = [my_df_dict[k] for k in my_df_dict.keys()]
    full_df = pd.concat(df_list,ignore_index=True)

    '''drop low AP amps'''
    for r in full_df.index:
        if full_df.loc[r,'AP amp'] <AP_cut:
            full_df.at[r,'AP amp'] = np.nan
    return full_df

In [11]:
### Git clone data tables for sorting ###
try: shutil.rmtree('./'+git_rep)
except: None
git_rep = 'Prelim_2023'
git_link = 'https://github.com/dtabuena/'+git_rep+'/'
!git clone $git_link
file_list = [f for f in [f for r,d,f in os.walk('./'+git_rep+'/Cleaned_DataSets/' )][0] if '.xlsx' in f]
print(file_list)
mpl.font_manager.fontManager.addfont('./'+git_rep+'/Cleaned_DataSets/arial.ttf')

Cloning into 'Prelim_2023'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 67 (delta 28), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (67/67), 14.85 MiB | 9.10 MiB/s, done.
Resolving deltas: 100% (28/28), done.
['CA1 main parameters_Clean.xlsx', 'DG Type II GC main parameters_Clean.xlsx', 'CA3 parameters for PCA_Clean.xlsx', 'DG Type I GC main parameters_Clean.xlsx']


In [12]:
# link = 'https://www.dropbox.com/sh/8gqut6sm8augo6p/AAD_jPpZFTFlaQlo6xOC5ENna?dl=0'
# file_list = get_files(link)
# file_list = [f for f in file_list if 'param' in f]
# _ = [print(f) for f in file_list]

In [13]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels

def MLR_FDR(full_df,cell_param_cols,key_list = ['genotype_key', 'age_key', 'combo_key' ]):
    MLRT_df= pd.DataFrame(index=cell_param_cols)
    for c in cell_param_cols:
        y_full = full_df[c]
        x_full = full_df[key_list]
        x_full = sm.add_constant(x_full)
        x_null = x_full['const']
        full_model = sm.OLS(y_full, x_full,missing = 'drop').fit()
        null_model = sm.OLS(y_full, x_null,missing = 'drop').fit()
        full_ll = full_model.llf
        null_ll = null_model.llf
        LR_statistic = -2*(null_ll-full_ll)
        p_val = scipy.stats.chi2.sf(LR_statistic, 2)
        MLRT_df.at[c,'Full Model log-likelihood'] = full_ll
        MLRT_df.at[c,'Null Model log-likelihood'] = null_ll
        MLRT_df.at[c,'LR statistic'] = LR_statistic
        MLRT_df.at[c,'p value'] = p_val
    FDR = statsmodels.stats.multitest.fdrcorrection(MLRT_df['p value'],alpha=0.05)
    MLRT_df['FDR corrected p'] = FDR[1]
    MLRT_df['Hit'] = FDR[0]
    return MLRT_df

In [14]:
def annotate_full_df(full_df, omit_cre=False,is_dg=False):
    if 'fAHP(1)' in full_df.columns:
        full_df['fAHP'] = full_df['fAHP(1)']
        full_df.drop(labels='fAHP(1)',axis=1,inplace=True)

    geno_list = full_df['type']
    geno_list = [g.replace(' 7-9mo',"") for g in geno_list]
    geno_list = [g.replace(' 17-19mo',"") for g in geno_list]
    geno_list = [g.replace('type_1 ',"") for g in geno_list]
    geno_list = [g.replace('type_2 ',"") for g in geno_list]
    full_df['genotype'] = geno_list
    full_df['genotype_key'] = [1 if 'E4' in g  else 0 for g in geno_list]

    age_list = full_df['type']
    age_list = [a.replace('apoE3-KI ',"") for a in age_list]
    age_list = [a.replace('apoE4-KI ',"") for a in age_list]
    age_list = [a.replace('fE4/Syn1-Cre- ',"") for a in age_list]
    age_list = [a.replace('fE4/Syn1-Cre+ ',"") for a in age_list]
    age_list = [a.replace('type_1 ',"") for a in age_list]
    age_list = [a.replace('type_2 ',"") for a in age_list]
    full_df['age'] = age_list
    full_df['age_key'] = [1 if '17' in a  else 0 for a in age_list]

    full_df['geno_age_inter'] = full_df['age_key']* 2 + full_df['genotype_key']
    key_list = ['genotype_key','age_key','geno_age_inter']

    if is_dg:
        dgt_list = full_df['dg_type']
        full_df['dgt_key'] = [1 if '2' in g  else 0 for g in geno_list]
        full_df['type_age_inter'] = full_df['age_key']* 2 + full_df['dgt_key']
        full_df['type_geno_inter'] = full_df['dgt_key']* 2 + full_df['genotype_key']
        full_df['geno_age_inter'] = full_df['age_key']* 2 + full_df['genotype_key']
        full_df['triple_inter'] = full_df['dgt_key']*4 +  full_df['age_key']* 2 + full_df['genotype_key']
        key_list = ['genotype_key','age_key','dgt_key','type_age_inter','type_geno_inter','geno_age_inter','triple_inter']

    if omit_cre: # omit cre
        drop_rows = [i for i in full_df.index if 'Cre' in full_df.loc[i,'type']]
        full_df.drop(drop_rows,axis = 0,inplace=True)
    return full_df, key_list

In [15]:
mlrt_folder = 'mlrt_folder'
try: os.makedirs(mlrt_folder)
except: None

#### CA3 Group Stats
_csv = [f for f in file_list if 'CA3' in f][0]
my_df_dict, xl_to_analyze = read_data_file(_csv)
full_df = dict_to_df(my_df_dict,AP_cut=60)
full_df,key_list = annotate_full_df(full_df, omit_cre='True')
labeling = ['cell','Cell','type','extra','age','dg_type','genotype'] + key_list
cell_param_cols = [c for c in full_df.columns if c not in labeling ]
ca3_MLRT_df = MLR_FDR(full_df,cell_param_cols,key_list)
ca3_MLRT_df.to_csv(mlrt_folder+'/Ca3_MaxLikeRatio.csv')

#### CA1 Group Stats
_csv = [f for f in file_list if 'CA1' in f][0]
my_df_dict, xl_to_analyze = read_data_file(_csv)
full_df = dict_to_df(my_df_dict,AP_cut=60)
full_df,key_list = annotate_full_df(full_df, omit_cre='True')
labeling = ['cell','Cell','type','extra','age','dg_type','genotype'] + key_list
cell_param_cols = [c for c in full_df.columns if c not in labeling ]
ca1_MLRT_df = MLR_FDR(full_df,cell_param_cols,key_list)
ca1_MLRT_df.to_csv(mlrt_folder+'/Ca1_MaxLikeRatio.csv')

#### DG Group Stats
_csv = [f for f in file_list if 'Type II' in f][0]
typeII_my_df_dict, xl_to_analyze = read_data_file(_csv)
typeII_df = dict_to_df(typeII_my_df_dict,AP_cut=60)
typeII_df['dg_type']='type_2'
for r in typeII_df.index: typeII_df.at[r,'type'] = 'type_2 '+ typeII_df.loc[r,'type']

_csv = [f for f in file_list if 'Type I ' in f][0]
typeI_my_df_dict, xl_to_analyze = read_data_file(_csv)
typeI_df = dict_to_df(typeI_my_df_dict,AP_cut=60)
typeI_df['dg_type']='type_1'
for r in typeI_df.index: typeI_df.at[r,'type'] = 'type_1 '+ typeI_df.loc[r,'type']

full_df = pd.concat([typeI_df,typeII_df],  axis=0, join='outer', ignore_index=True)
full_df,key_list = annotate_full_df(full_df, omit_cre='True',is_dg=True)


labeling = ['cell','Cell','type','extra','age','dg_type','genotype'] + key_list
cell_param_cols = [c for c in full_df.columns if c not in labeling ]
dg_MLRT_df = MLR_FDR(full_df,cell_param_cols,key_list)
dg_MLRT_df.to_csv(mlrt_folder+'/DG_MaxLikeRatio.csv')

In [16]:
from google import colab
colab.files.download( mlrt_folder+'/DG_MaxLikeRatio.csv' )
colab.files.download( mlrt_folder+'/Ca3_MaxLikeRatio.csv' )
colab.files.download( mlrt_folder+'/Ca1_MaxLikeRatio.csv' )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>